## Creating Excel files for missing and zero values data 

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import os

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# List of all device IDs you want to process
device_ids = [1202240026,1202240025,1203240081,1202240011,1202240027,1203240076,1203240078,1203240075,1201240077,
    1201240072,1203240079,1201240079 ,1201240085 ,1203240083 ,1203240073 ,1203240074,1201240076,1212230160 ,
    1202240009 ,1202240008,1201240073,1203240080,1201240074,1203240077,1203240082,1202240029,1202240028,1202240010,1202240012]

# Directory to save Excel files
save_dir = r"C:\Users\abhis\Data_Validation"
os.makedirs(save_dir, exist_ok=True)

# Connect to the database
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")

    cursor = conn.cursor()

    # SQL query
    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s;
    """

    # Loop over each ID
    for id_to_fetch in device_ids:
        print(f"\nProcessing Device ID: {id_to_fetch}")
        cursor.execute(query, (id_to_fetch,))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2", "voc", "temp", "humidity", "battery", "viral_index"])
            df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
            df = df.dropna(subset=['datetime'])
            df['date'] = df['datetime'].dt.date

            all_dates = pd.date_range(df['date'].min(), df['date'].max(), freq='D').date
            existing_dates = df['date'].unique()
            missing_dates = [date for date in all_dates if date not in existing_dates]

            zero_data_dates = []
            for date in df['date'].unique():
                day_data = df[df['date'] == date]
                if (day_data[['pm25', 'pm10', 'aqi', 'co2', 'voc', 'temp', 'humidity', 'battery', 'viral_index']] == 0).all().all():
                    zero_data_dates.append(date)

            df1 = pd.DataFrame(missing_dates, columns=['Missing Dates'])
            df2 = pd.DataFrame(zero_data_dates, columns=['Zero Values'])

            df_con = pd.concat([df1, df2], axis=1)
            file_path = os.path.join(save_dir, f"{id_to_fetch}.xlsx")
            df_con.to_excel(file_path, index=False)
            print(f"Data for Device ID {id_to_fetch} saved to {file_path}")
        else:
            print(f"No data found for Device ID: {id_to_fetch}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


### Including Percentage of Presence of data for a particular day in Excel files (End datetime is define explicitly)

In [7]:
import mysql.connector
import pandas as pd
import numpy as np
import os

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# List of all device IDs you want to process
device_ids = [  
              1201240085, 1203240083, 1203240073, 1203240074, 1201240076, 1212230160,
              1202240009, 1202240008, 
              1201240073, 1203240080, ]

# Directory to save Excel files
save_dir = r"C:\Users\abhis\Data_Validation_05-08"
os.makedirs(save_dir, exist_ok=True)

# Connect to the database
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")

    cursor = conn.cursor()

    # SQL query
    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s;
    """

    # Loop over each ID
    for id_to_fetch in device_ids:
        print(f"\nProcessing Device ID: {id_to_fetch}")
        cursor.execute(query, (id_to_fetch,))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2", "voc", "temp", "humidity", "battery", "viral_index"])
            df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
            df = df.dropna(subset=['datetime'])
            df['date'] = df['datetime'].dt.date
            # df.to_csv('testing_data_validation.csv')

            all_dates = pd.date_range(df['date'].min(), df['date'].max(), freq='D').date
            existing_dates = df['date'].unique()
            missing_dates = [date for date in all_dates if date not in existing_dates]

            # Finding Zero value data.
            zero_data_dates = []
            for date in df['date'].unique():
                day_data = df[df['date'] == date]
                if (day_data[['pm25', 'pm10', 'aqi', 'co2', 'voc', 'temp', 'humidity', 'battery', 'viral_index']] == 0).all().all():
                    zero_data_dates.append(date)

            # Calculate data presence percentage for each day
            data_presence = []
            for date in df['date'].unique():
                day_data = df[df['date'] == date]
                presence_percentage = (len(day_data) / 1440) * 100  # 1440 minutes in a day
                data_presence.append({'Date': date, 'Data Presence (%)': presence_percentage})

            start_datetime = df['datetime'].min()
            forced_end_datetime = pd.to_datetime("2025-03-31 23:59")
            
            # Adjust end_datetime if data is shorter than expected period
            end_datetime = forced_end_datetime
            
            total_minutes = int((end_datetime - start_datetime).total_seconds() / 60) + 1
            expected_records = total_minutes
            actual_records = len(df)
            overall_data_quality = (actual_records / expected_records) * 100


            
            df1 = pd.DataFrame(missing_dates, columns=['Missing Dates'])
            df2 = pd.DataFrame(zero_data_dates, columns=['Zero Values'])
            df3 = pd.DataFrame(data_presence)
            quality_df = pd.DataFrame({
                'Metric': ['Device ID', 'Start Datetime', 'End Datetime', 'Expected Records', 'Actual Records', 'Data Quality (%)'],
                'Value': [id_to_fetch, start_datetime, end_datetime, expected_records, actual_records, overall_data_quality]
            })
            

            df_con = pd.concat([df1, df2, df3,quality_df], axis=1)
            file_path = os.path.join(save_dir, f"{id_to_fetch}.xlsx")
            df_con.to_excel(file_path, index=False)
            print(f"Data for Device ID {id_to_fetch} saved to {file_path}")
        else:
            print(f"No data found for Device ID: {id_to_fetch}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


Database connection successful.

Processing Device ID: 1201240085
Data for Device ID 1201240085 saved to C:\Users\abhis\Data_Validation_05-08\1201240085.xlsx

Processing Device ID: 1203240083
Data for Device ID 1203240083 saved to C:\Users\abhis\Data_Validation_05-08\1203240083.xlsx

Processing Device ID: 1203240073
Data for Device ID 1203240073 saved to C:\Users\abhis\Data_Validation_05-08\1203240073.xlsx

Processing Device ID: 1203240074
Data for Device ID 1203240074 saved to C:\Users\abhis\Data_Validation_05-08\1203240074.xlsx

Processing Device ID: 1201240076
Data for Device ID 1201240076 saved to C:\Users\abhis\Data_Validation_05-08\1201240076.xlsx

Processing Device ID: 1212230160
Data for Device ID 1212230160 saved to C:\Users\abhis\Data_Validation_05-08\1212230160.xlsx

Processing Device ID: 1202240009
Data for Device ID 1202240009 saved to C:\Users\abhis\Data_Validation_05-08\1202240009.xlsx

Processing Device ID: 1202240008
Data for Device ID 1202240008 saved to C:\Users\abhi

In [19]:
import mysql.connector
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# Device IDs to process
device_ids = [  
    1202240026, 1202240025, 1203240081, 1202240011, 1202240027, 1203240076,
    1203240078, 1203240075, 1201240077, 1201240072, 1203240079, 1201240079,
    1201240085, 1203240083, 1203240073, 1203240074, 1201240076, 1212230160,
    1202240009, 1202240008, 1201240073, 1203240080, 1201240074, 1203240077,
    1203240082, 1202240029, 1202240028, 1202240010, 1202240012
]

# Output directory
save_dir = r"C:\Users\abhis\Data_Validation_tillMarch_2024_06-08"
os.makedirs(save_dir, exist_ok=True)

# Forced end datetime
FORCED_END_DATETIME = pd.to_datetime("2025-03-31 23:59:59")

# Connect to the database
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")

    cursor = conn.cursor()

    # SQL query
    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s;
    """

    for id_to_fetch in device_ids:
        print(f"\nProcessing Device ID: {id_to_fetch}")
        cursor.execute(query, (id_to_fetch,))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2", "voc", "temp", "humidity", "battery", "viral_index"])
            
            # Parse datetime and filter by FORCED_END_DATETIME
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            df = df.dropna(subset=['datetime'])
            df = df[df['datetime'] <= FORCED_END_DATETIME]

            # Optional: print max datetime to confirm
            print("Latest datetime in filtered data:", df['datetime'].max())

            # Extract date
            df['date'] = df['datetime'].dt.date

            # Determine all possible dates and missing dates
            all_dates = pd.date_range(df['date'].min(), FORCED_END_DATETIME.date(), freq='D').date
            existing_dates = df['date'].unique()
            missing_dates = [date for date in all_dates if date not in existing_dates]

            # Initialize
            zero_data_dates = []
            data_presence = []
            filtered_df = pd.DataFrame()  # Will hold only non-zero rows

            for date in existing_dates:
                day_data = df[df['date'] == date]

                # Check if ALL rows for the day are zero
                if (day_data[['pm25', 'pm10', 'aqi', 'co2', 'voc', 'temp', 'humidity', 'battery', 'viral_index']] == 0).all().all():
                    zero_data_dates.append(date)
                else:
                    # Exclude rows where all monitored values are zero
                    valid_day_data = day_data[
                        ~(day_data[['pm25', 'pm10', 'aqi', 'co2', 'voc', 'temp', 'humidity', 'battery', 'viral_index']] == 0).all(axis=1)
                    ]
                    presence_percentage = (len(valid_day_data) / 1440) * 100
                    data_presence.append({'Date': date, 'Data Presence (%)': presence_percentage})

                    # Add to filtered_df for quality calculation
                    filtered_df = pd.concat([filtered_df, valid_day_data])

            # Data quality calculation
            if not filtered_df.empty:
                start_datetime = filtered_df['datetime'].min()

                if start_datetime > FORCED_END_DATETIME:
                    expected_records = 0
                    actual_records = 0
                    overall_data_quality = 0
                    end_datetime = start_datetime
                else:
                    end_datetime = FORCED_END_DATETIME
                    total_minutes = int((end_datetime - start_datetime).total_seconds() / 60) + 1
                    expected_records = total_minutes
                    actual_records = len(filtered_df)
                    overall_data_quality = (actual_records / expected_records) * 100
            else:
                start_datetime = None
                end_datetime = FORCED_END_DATETIME
                expected_records = 0
                actual_records = 0
                overall_data_quality = 0

            # Prepare output data
            df1 = pd.DataFrame(missing_dates, columns=['Missing Dates'])
            df2 = pd.DataFrame(zero_data_dates, columns=['Zero Values'])
            df3 = pd.DataFrame(data_presence)
            quality_df = pd.DataFrame({
                'Metric': ['Device ID', 'Start Datetime', 'End Datetime', 'Expected Records', 'Actual Records', 'Data Quality (%)'],
                'Value': [id_to_fetch, start_datetime, end_datetime, expected_records, actual_records, overall_data_quality]
            })

            # Combine all into one Excel sheet (aligned on index)
            combined_df = pd.concat([df1, df2, df3, quality_df], axis=1)

            # Save to Excel
            file_path = os.path.join(save_dir, f"{id_to_fetch}.xlsx")
            combined_df.to_excel(file_path, index=False)
            print(f"Data for Device ID {id_to_fetch} saved to {file_path}")
        else:
            print(f"No data found for Device ID: {id_to_fetch}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


Database connection successful.

Processing Device ID: 1202240026
Latest datetime in filtered data: 2025-03-31 23:59:02
Data for Device ID 1202240026 saved to C:\Users\abhis\Data_Validation_tillMarch_2024_06-08\1202240026.xlsx

Processing Device ID: 1202240025
Latest datetime in filtered data: 2025-03-31 23:59:02
Data for Device ID 1202240025 saved to C:\Users\abhis\Data_Validation_tillMarch_2024_06-08\1202240025.xlsx

Processing Device ID: 1203240081
Latest datetime in filtered data: 2025-03-31 23:59:02
Data for Device ID 1203240081 saved to C:\Users\abhis\Data_Validation_tillMarch_2024_06-08\1203240081.xlsx

Processing Device ID: 1202240011
Latest datetime in filtered data: 2025-03-31 23:59:02
Data for Device ID 1202240011 saved to C:\Users\abhis\Data_Validation_tillMarch_2024_06-08\1202240011.xlsx

Processing Device ID: 1202240027
Latest datetime in filtered data: 2025-03-31 23:59:02
Data for Device ID 1202240027 saved to C:\Users\abhis\Data_Validation_tillMarch_2024_06-08\120224002

In [5]:
import mysql.connector
import pandas as pd
import numpy as np
import os

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# List of all device IDs you want to process
# device_ids = [
#     1202240026, 1202240025, 1203240081, 1202240011, 1202240027, 1203240076,
#     1203240078, 1203240075, 1201240077, 1201240072, 1203240079, 1201240079,
#     1201240085, 1203240083, 1203240073, 1203240074, 1201240076, 1212230160,
#     1202240009, 1202240008, 1201240073, 1203240080, 1201240074, 1203240077,
#     1203240082, 1202240029, 1202240028, 1202240010, 1202240012
# ]

device_ids = [
    1201240075,1201240085

]

# Directory to save Excel files
save_dir = r"C:\Users\abhis\Data_Validation_01-08"
os.makedirs(save_dir, exist_ok=True)

# Connect to the database
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")

    cursor = conn.cursor()

    # SQL query
    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s;
    """

    # Loop over each ID
    for id_to_fetch in device_ids:
        print(f"\nProcessing Device ID: {id_to_fetch}")
        cursor.execute(query, (id_to_fetch,))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=[
                "id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2",
                "voc", "temp", "humidity", "battery", "viral_index"
            ])
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            df = df.dropna(subset=['datetime'])
            df['date'] = df['datetime'].dt.date

            if df.empty:
                print(f"No valid datetime entries for Device ID: {id_to_fetch}")
                continue

            # Missing Dates
            all_dates = pd.date_range(df['date'].min(), df['date'].max(), freq='D').date
            existing_dates = df['date'].unique()
            missing_dates = [date for date in all_dates if date not in existing_dates]
            df1 = pd.DataFrame(missing_dates, columns=['Missing Dates'])
            if df1.empty:
                df1 = pd.DataFrame({'Missing Dates': ['None']})

            # Zero Value Dates
            zero_data_dates = []
            for date in df['date'].unique():
                day_data = df[df['date'] == date]
                if (day_data[['pm25', 'pm10', 'aqi', 'co2', 'voc', 'temp', 'humidity', 'battery', 'viral_index']] == 0).all().all():
                    zero_data_dates.append(date)
            df2 = pd.DataFrame(zero_data_dates, columns=['Zero Values'])
            if df2.empty:
                df2 = pd.DataFrame({'Zero Values': ['None']})

            # Daily Data Presence
            data_presence = []
            for date in df['date'].unique():
                day_data = df[df['date'] == date]
                presence_percentage = (len(day_data) / 1440) * 100
                data_presence.append({'Date': date, 'Data Presence (%)': presence_percentage})
            df3 = pd.DataFrame(data_presence)
            if df3.empty:
                df3 = pd.DataFrame({'Date': ['None'], 'Data Presence (%)': [0]})

            # Overall Data Quality
            start_datetime = df['datetime'].min()
            end_datetime = df['datetime'].max()
            total_minutes = int((end_datetime - start_datetime).total_seconds() / 60) + 1
            expected_records = total_minutes
            actual_records = len(df)
            overall_data_quality = (actual_records / expected_records) * 100

            quality_df = pd.DataFrame({
                'Metric': ['Device ID', 'Start Datetime', 'End Datetime', 'Expected Records', 'Actual Records', 'Data Quality (%)'],
                'Value': [id_to_fetch, start_datetime, end_datetime, expected_records, actual_records, overall_data_quality]
            })

            # Save to Excel
            file_path = os.path.join(save_dir, f"{id_to_fetch}.xlsx")
            with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                df1.to_excel(writer, sheet_name='Missing Dates', index=False)
                df2.to_excel(writer, sheet_name='Zero Values', index=False)
                df3.to_excel(writer, sheet_name='Daily Presence (%)', index=False)
                quality_df.to_excel(writer, sheet_name='Overall Quality', index=False)

            print(f"Data for Device ID {id_to_fetch} saved to {file_path}")
        else:
            print(f"No data found for Device ID: {id_to_fetch}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


Database connection successful.

Processing Device ID: 1201240075
Data for Device ID 1201240075 saved to C:\Users\abhis\Data_Validation_01-08\1201240075.xlsx

Processing Device ID: 1201240085
Data for Device ID 1201240085 saved to C:\Users\abhis\Data_Validation_01-08\1201240085.xlsx

Connection closed.


### for caclulating the data percentage from the first available data points in april 2024 months and first available data point in march 2025 month


In [21]:
import mysql.connector
import pandas as pd
import os
from datetime import datetime

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# Device IDs
device_ids = [
    1202240026, 1202240025, 1203240081, 1202240011, 1202240027, 1203240076,
    1203240078, 1203240075, 1201240077, 1201240072, 1203240079, 1201240079,
    1201240085, 1203240083, 1203240073, 1203240074, 1201240076, 1212230160,
    1202240009, 1202240008, 1201240073, 1203240080, 1201240074, 1203240077,
    1203240082, 1202240029, 1202240028, 1202240010, 1202240012, 1201240075,
    1201240078, 1203240072
]

# Save directory and output file
save_dir = r"C:\Users\abhis\Data_Vaidation_April_March"
os.makedirs(save_dir, exist_ok=True)
output_csv_path = os.path.join(save_dir, "All_Device_Data_Quality.csv")

# April 1, 2024 to March 31, 2025
filter_start = datetime(2024, 4, 1)
filter_end = datetime(2025, 3, 31, 23, 59, 59)

# List to store data for all devices
all_device_results = []

# Connect and process
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")
    cursor = conn.cursor()

    # Query with date range
    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s AND datetime BETWEEN %s AND %s;
    """

    for id_to_fetch in device_ids:
        print(f"\nProcessing Device ID: {id_to_fetch}")
        cursor.execute(query, (id_to_fetch, filter_start, filter_end))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=[
                "id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2", "voc",
                "temp", "humidity", "battery", "viral_index"
            ])
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            df = df.dropna(subset=['datetime'])

            actual_start = df['datetime'].min()
            actual_end = df['datetime'].max()
            df = df[(df['datetime'] >= actual_start) & (df['datetime'] <= actual_end)]

            total_minutes = int((actual_end - actual_start).total_seconds() / 60) + 1
            expected_records = total_minutes
            actual_records = len(df)
            overall_data_quality = (actual_records / expected_records) * 100

            all_device_results.append({
                'Device ID': id_to_fetch,
                'Start Datetime': actual_start,
                'End Datetime': actual_end,
                'Expected Records': expected_records,
                'Actual Records': actual_records,
                'Data Quality (%)': round(overall_data_quality, 2)
            })
        else:
            print(f"No data found for Device ID: {id_to_fetch}")

    # Save all results to a single CSV file
    final_df = pd.DataFrame(all_device_results)
    final_df.to_csv(output_csv_path, index=False)
    print(f"\nCombined data quality saved to: {output_csv_path}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


Database connection successful.

Processing Device ID: 1202240026

Processing Device ID: 1202240025

Processing Device ID: 1203240081

Processing Device ID: 1202240011

Processing Device ID: 1202240027

Processing Device ID: 1203240076

Processing Device ID: 1203240078

Processing Device ID: 1203240075

Processing Device ID: 1201240077

Processing Device ID: 1201240072

Processing Device ID: 1203240079

Processing Device ID: 1201240079

Processing Device ID: 1201240085

Processing Device ID: 1203240083

Processing Device ID: 1203240073

Processing Device ID: 1203240074

Processing Device ID: 1201240076

Processing Device ID: 1212230160

Processing Device ID: 1202240009

Processing Device ID: 1202240008

Processing Device ID: 1201240073

Processing Device ID: 1203240080

Processing Device ID: 1201240074

Processing Device ID: 1203240077

Processing Device ID: 1203240082

Processing Device ID: 1202240029

Processing Device ID: 1202240028

Processing Device ID: 1202240010

Processing Devi

### For Calculating the data percentage for overall time period

In [8]:
import mysql.connector
import pandas as pd
import os

# Database credentials
host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

# List of all device IDs to process
device_ids = [1201240075, 1201240085]

# Directory to save the CSV file
save_dir = r"C:\Users\abhis\Data_Validation_01-08"
os.makedirs(save_dir, exist_ok=True)

# Output list to collect all device data quality
output_data = []

# Connect to the database
conn = None
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")
    cursor = conn.cursor()

    # SQL query
    query = """
    SELECT datetime
    FROM reading_db
    WHERE deviceID = %s;
    """

    for device_id in device_ids:
        print(f"\nProcessing Device ID: {device_id}")
        cursor.execute(query, (device_id,))
        rows = cursor.fetchall()

        if rows:
            df = pd.DataFrame(rows, columns=["datetime"])
            df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
            df = df.dropna(subset=['datetime'])

            if df.empty:
                print(f"No valid datetime entries for Device ID: {device_id}")
                continue

            start_datetime = df['datetime'].min()
            end_datetime = df['datetime'].max()
            total_minutes = int((end_datetime - start_datetime).total_seconds() / 60) + 1
            expected_records = total_minutes
            actual_records = len(df)
            overall_data_quality = (actual_records / expected_records) * 100

            output_data.append({
                'Device ID': device_id,
                'Start Datetime': start_datetime,
                'End Datetime': end_datetime,
                'Expected Records': expected_records,
                'Actual Records': actual_records,
                'Data Quality (%)': round(overall_data_quality, 2)
            })

        else:
            print(f"No data found for Device ID: {device_id}")

    # Save all device results into a single CSV
    output_df = pd.DataFrame(output_data)
    output_csv_path = os.path.join(save_dir, "device_data_quality.csv")
    output_df.to_csv(output_csv_path, index=False)
    print(f"\nOverall data quality saved to: {output_csv_path}")

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("\nConnection closed.")


Database connection successful.

Processing Device ID: 1201240075

Processing Device ID: 1201240085

Overall data quality saved to: C:\Users\abhis\Data_Validation_01-08\device_data_quality.csv

Connection closed.
